<a href="https://colab.research.google.com/github/AkhilaGHub/Pregnancy-Insights-AI-Powered-Personalized-Care-with-Retrieval-Augmented-Generation/blob/main/Pregnancy_Insights_AI_Powered_Personalized_Care_with_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart: Querying PDF With Astra and LangChain

### A question-answering demo using Astra DB and LangChain, powered by Vector Search

#### Pre-requisites:

You need a **_Serverless Cassandra with Vector Search_** database on [Astra DB](https://astra.datastax.com) to run this demo. As outlined in more detail [here](https://docs.datastax.com/en/astra-serverless/docs/vector-search/quickstart.html#_prepare_for_using_your_vector_database), you should get a DB Token with role _Database Administrator_ and copy your Database ID: these connection parameters are needed momentarily.

You also need an [OpenAI API Key](https://cassio.org/start_here/#llm-access) for this demo to work.

#### What you will do:

- Setup: import dependencies, provide secrets, create the LangChain vector store;
- Run a Question-Answering loop retrieving the relevant headlines and having an LLM construct the answer.

Install the required dependencies:

In [ ]:
pip install torch

In [ ]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 704.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 546.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15

In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00


Import the packages you'll need:

In [ ]:
# LangChain components to use
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for dataset retrieval with Hugging Face
from datasets import load_dataset

# With CassIO, the engine powering the Astra DB integration in LangChain,
# you will also initialize the DB connection:
import cassio

In [ ]:

!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.7 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

### Setup

#### Provide your secrets:

Replace the following with your Astra DB connection details and your OpenAI API key:

In [ ]:
ASTRA_DB_APPLICATION_TOKEN = "" # enter the "AstraCS:..." string found in in your Token JSON file
ASTRA_DB_ID = "" # enter your Database ID

OPENAI_API_KEY = "" # enter your OpenAI key

In [ ]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/content/Pregnancy Care.pdf')

In [ ]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
raw_text

'Pregnancy\nCare\nPregnancy\nis\nthe\nperiod\nduring\nwhich\na\nfertilized\negg\ndevelops\ninto\na\nfetus\ninside\na\nwoman\'s\nwomb.\nCarrying\na\nbaby\nis\nthe\nmost\nrewarding\nexperience\na\nwoman\ncan\nenjoy .\nIt\ntypically\nlasts\nabout\n40\nweeks,\ncounting\nfrom\nthe\nfirst\nday\nof\nthe\nlast\nmenstrual\nperiod\n(LMP)\nto\ndelivery .\nStages\nof\nPregnancy\n(Trimesters)\n●\nFirst\nTrimester\n(Weeks\n1-12)\n:\nMajor\norgans\nand\nbody\nsystems\nbegin\nto\nform.\nThe\nembryo\ntransitions\ninto\na\nfetus\nby\nthe\nend\nof\nthe\n8th\nweek.\nSymptoms:\nNausea,\nfatigue,\nbreast\ntenderness,\nfrequent\nurination,\nand\nmood\nswings.\n●\nSecond\nTrimester\n(Weeks\n13-26)\n:The\nfetus\ngrows\nrapidly ,\nand\nthe\nmother\nmay\nstart\nto\nfeel\nfetal\nmovements\n(quickening).\nSymptoms:\nReduced\nnausea,\nincreased\nenergy,\nvisible\npregnancy\nbump,\nand\npossible\nswelling\nin\nlegs\nand\nankles.\n●\nThird\nTrimester\n(Weeks\n27-40)\n:\nThe\nfetus\ngains\nweight,\nlungs\nmature,\nand

Initialize the connection to your database:

_(do not worry if you see a few warnings, it's just that the drivers are chatty about negotiating protocol versions with the DB.)_

In [ ]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <LibevConnection(133195532208352) 9c021902-c44c-465a-84a6-88866f77b2b1-us-east1.db.astra.datastax.com:29042:a4208368-c3be-4134-8a22-b8080244a1cb> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Create the LangChain embedding and LLM objects for later usage:

In [ ]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

<ipython-input-12-9c33d76eff6b>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(openai_api_key=OPENAI_API_KEY)
<ipython-input-12-9c33d76eff6b>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


Create your LangChain vector store ... backed by Astra DB!

In [ ]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[:50]

["Pregnancy\nCare\nPregnancy\nis\nthe\nperiod\nduring\nwhich\na\nfertilized\negg\ndevelops\ninto\na\nfetus\ninside\na\nwoman's\nwomb.\nCarrying\na\nbaby\nis\nthe\nmost\nrewarding\nexperience\na\nwoman\ncan\nenjoy .\nIt\ntypically\nlasts\nabout\n40\nweeks,\ncounting\nfrom\nthe\nfirst\nday\nof\nthe\nlast\nmenstrual\nperiod\n(LMP)\nto\ndelivery .\nStages\nof\nPregnancy\n(Trimesters)\n●\nFirst\nTrimester\n(Weeks\n1-12)\n:\nMajor\norgans\nand\nbody\nsystems\nbegin\nto\nform.\nThe\nembryo\ntransitions\ninto\na\nfetus\nby\nthe\nend\nof\nthe\n8th\nweek.\nSymptoms:\nNausea,\nfatigue,\nbreast\ntenderness,\nfrequent\nurination,\nand\nmood\nswings.\n●\nSecond\nTrimester\n(Weeks\n13-26)\n:The\nfetus\ngrows\nrapidly ,\nand\nthe\nmother\nmay\nstart\nto\nfeel\nfetal\nmovements\n(quickening).\nSymptoms:\nReduced\nnausea,\nincreased\nenergy,\nvisible\npregnancy\nbump,\nand\npossible\nswelling\nin\nlegs\nand\nankles.\n●\nThird",
 'grows\nrapidly ,\nand\nthe\nmother\nmay\nstart\nto\nfeel\nfetal\nmovements

### Load the dataset into the vector store



In [ ]:

astra_vector_store.add_texts(texts[:50])

print("Inserted %i headlines." % len(texts[:50]))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 50 headlines.


### Run the QA cycle

Simply run the cells and ask a question -- or `quit` to stop. (you can also stop execution with the "▪" button on the top toolbar)

Here are some suggested questions:
- _What is the current GDP?_
- _How much the agriculture target will be increased to and what the focus will be_


In [ ]:
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): what are stages of pregnancy

QUESTION: "what are stages of pregnancy"


ANSWER: "The stages of pregnancy are divided into three trimesters: the first trimester (weeks 1-12), the second trimester (weeks 13-26), and the third trimester (weeks 27-40). Each trimester has different developments and symptoms for both the mother and the fetus."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9327] "Pregnancy
Care
Pregnancy
is
the
period
during
which
a
fertilized
egg
develops
into
a ..."
    [0.9150] "grows
rapidly ,
and
the
mother
may
start
to
feel
fetal
movements
(quickening).
Sympt ..."
    [0.9122] "swimming,
prenatal
yoga,
and
light
strength
training.
These
help
in
maintaining
fitn ..."
    [0.9078] "her
to
the
PHC-Medical
Officer.
●
Routine
Checkups
and
Screenings
:
1.
Initial
Visit ..."

What's your next question (or type 'quit' to exit): what are trimesters?

QUESTION: "what are trimesters?"


ANSWER: "Trimesters refer to the three distinct stages of pregnancy, divided into first, second, and third trimesters. Each trimester has its own unique developments and symptoms."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9101] "Pregnancy
Care
Pregnancy
is
the
period
during
which
a
fertilized
egg
develops
into
a ..."
    [0.9004] "swimming,
prenatal
yoga,
and
light
strength
training.
These
help
in
maintaining
fitn ..."
    [0.8917] "trimester
if
possible,
as
this
is
the
most
critical
period
for
fetal
development.
Co ..."
    [0.8899] "grows
rapidly ,
and
the
mother
may
start
to
feel
fetal
movements
(quickening).
Sympt ..."

What's your next question (or type 'quit' to exit): what is prenatal care

QUESTION: "what is prenatal care"


...

    [0.8796] "her
to
the
PHC-Medical
Officer.
●
Routine
Checkups
and
Screenings
:
1.
Initial
Visit ..."
    [0.8780] "pregnancy
tests,
blood
tests,
and
an
ultrasound
to
confirm
the
gestational
age
and
c ..."
    [0.8766] "rate
of
the
district
is
20
per
1000
population,
then,
100
children
will
be
born
for
 ..."
    [0.8758] "woman
with
PIH,
pre-eclampsia
or
imminent
eclampsia
requires
hospitalization
and
sup ..."

What's your next question (or type 'quit' to exit): can you brief me on nicu?

QUESTION: "can you brief me on nicu?"


ANSWER: "NICU stands for Neonatal Intensive Care Unit and it is a specialized unit in a hospital that provides intensive medical care for newborn babies who are born prematurely or have serious health conditions. These units are equipped with advanced medical technology and a highly trained medical team to monitor and treat the newborns. The goal of the NICU is to provide the best possible care for the baby while supporting the family during a difficult and stressful time."

FIRST DOCUMENTS BY RELEVANCE:


    [0.8921] "her
to
the
PHC-Medical
Officer.
●
Routine
Checkups
and
Screenings
:
1.
Initial
Visit ..."
    [0.8823] "or
intervention
is
recommended
at
that
point
of
time.
Advise
the
woman
to
go
in
for
 ..."
    [0.8820] "commonly
known
to
try
to
eat
healthy
while
avoiding
certain
foods,
exercise,
and
tak ..."
    [0.8798] "pregnancy
tests,
blood
tests,
and
an
ultrasound
to
confirm
the
gestational
age
and
c ..."

What's your next question (or type 'quit' to exit): what are general pregnancy complications?

QUESTION: "what are general pregnancy complications?"


ANSWER: "Some general pregnancy complications include high-risk activities, common discomforts like nausea and vomiting, and physical changes like back pain. It is important for pregnant women to avoid high-impact sports, heavy lifting, and activities that increase the risk of falls or abdominal trauma. They should also manage common discomforts such as morning sickness and back pain through strategies like small, frequent meals and low-impact exercise. It is also important for pregnant women to monitor for symptoms of more serious complications, such as severe anemia, jaundice, high blood pressure, and preeclampsia, and seek medical attention if necessary. Mental health considerations, such as managing stress and seeking support for emotional and psychological changes, are also important during pregnancy."

FIRST DOCUMENTS BY RELEVANCE:


    [0.9117] "swimming,
prenatal
yoga,
and
light
strength
training.
These
help
in
maintaining
fitn ..."
    [0.9072] "Pregnant
Woman
Refer
the
pregnant
woman
to
24-hour
PHC/FRU
for
further
examination
i ..."
    [0.9052] "by
other
symptoms
like
vaginal
bleeding,
consult
your
healthcare
provider
as
it
coul ..."
    [0.9027] "called
hyperemesis
gravidarum.
This
requires
medical
attention,
as
it
can
impact
bot ..."

What's your next question (or type 'quit' to exit): quit


In [ ]:
from sklearn.metrics import f1_score
import numpy as np

def exact_match(pred, true):
    return int(pred.strip().lower() == true.strip().lower())

def precision_recall_f1(pred, true):
    pred_tokens = pred.lower().split()
    true_tokens = true.lower().split()

    common_tokens = set(pred_tokens) & set(true_tokens)

    if len(common_tokens) == 0:
        return 0, 0, 0

    precision = len(common_tokens) / len(pred_tokens)
    recall = len(common_tokens) / len(true_tokens)
    f1 = 2 * precision * recall / (precision + recall)

    return precision, recall, f1

# Example Usage:
generated_answers = ["Trimesters refer to the three distinct stages of pregnancy, divided into first, second, and third trimesters. Each trimester has its own unique developments and symptoms.", "The condition of extreme nausea and vomiting that can impact both the mother and baby is called as Hyperemesis gravidarum. It is a severe form of morning sickness that requires medical attention during pregnancy.", " Anxiety during pregnancy is common, with many women experiencing worries about theirhealth, the baby’s health, and the changes that come with motherhood. Strategies for Managing Anxiety: Education, Positive Affirmations, Limiting Media Exposure, Mindfulness and Relaxation Techniques, Establish a daily routine that includes time for relaxation, hobbies, and activities that bring joy,Sharing experiences with other pregnant women can normalize your feelings and offer practical advice","Sleeping on your stomach is generally not recommended during pregnancy, as it can put added pressure on your growing belly and potentially restrict blood flow to the uterus. Doctors may advise against it after a certain point in pregnancy, and it may also increase the risk of stillbirth. It is generally recommended to sleep on your side, either the left or right, to promote optimal blood flow to the baby.", "The two types of delivery are vaginal delivery and cesarean section.", "Multiple pregnancies refer to a situation where a woman is carrying more than one fetus, such as twins, triplets, or higher order multiples.","NICU stands for Neonatal Intensive Care Unit and it is a specialized unit in a hospital that provides intensive medical care for newborn babies who are born prematurely or have serious health conditions. These units are equipped with advanced medical technology and a highly trained medical team to monitor and treat the newborns. The goal of the NICU is to provide the best possible care for the baby while supporting the family during a difficult and stressful time."
]
reference_answers = ["A trimester is divided into first, second, and third trimesters where first trimester lasting about 12-14 weeks, second trimester lasts about 13-26 weeks, third semester lasting about 27-36 weeks.","The condition of extreme nausea and vomiting that can impact both the mother and baby is called as Hyperemesis gravidarum. It is a severe form of morning sickness that requires medical attention during pregnancy"," Anxiety during pregnancy is common, with many women experiencing worries about theirhealth, the baby’s health, and the changes that come with motherhood. Strategies for Managing Anxiety: Education, Positive Affirmations, Limiting Media Exposure, Mindfulness and Relaxation Techniques, Establish a daily routine that includes time for relaxation, hobbies, and activities that bring joy,Sharing experiences with other pregnant women can normalize your feelings and offer practical advice","Sleeping on your stomach is generally not recommended during pregnancy, as it can put added pressure on your growing belly and potentially restrict blood flow to the uterus. Doctors may advise against it after a certain point in pregnancy, and it may also increase the risk of stillbirth. It is generally recommended to sleep , either the left or right, to promote optimal blood flow to the baby.", "The two types of delivery are vaginal delivery and cesarean section.", "Multiple pregnancies refer to a situation where a woman is carrying more than one fetus, such as twins, triplets, or higher order multiples.","NICU stands for Neonatal Intensive Care Unit and it is a specialized unit in a hospital that provides intensive medical care for newborn babies who are born prematurely or have serious health conditions. These units are equipped with advanced medical technology and a highly trained medical team to monitor and treat the newborns. The goal of the NICU is to provide the best possible care for the baby while supporting the family during a difficult and stressful time."]

exact_matches = [exact_match(pred, true) for pred, true in zip(generated_answers, reference_answers)]
f1_scores = [precision_recall_f1(pred, true)[2] for pred, true in zip(generated_answers, reference_answers)]

print("Exact Match Accuracy: ", np.mean(exact_matches))
print("Average F1 Score: ", np.mean(f1_scores))


Exact Match Accuracy:  0.5714285714285714
Average F1 Score:  0.7471383907605136


In [ ]:
pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=78269314ea2ad988b2ba80786a70e8b3bd9bc613d48eb0c0275326b13d5d6429
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

def rouge_scores(pred, true):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(true, pred)
    return scores

# Example Usage:
for pred, true in zip(generated_answers, reference_answers):
    scores = rouge_scores(pred, true)
    print(f"ROUGE-1: {scores['rouge1'].fmeasure}, ROUGE-2: {scores['rouge2'].fmeasure}, ROUGE-L: {scores['rougeL'].fmeasure}")


ROUGE-1: 0.3157894736842105, ROUGE-2: 0.21818181818181817, ROUGE-L: 0.3157894736842105
ROUGE-1: 1.0, ROUGE-2: 1.0, ROUGE-L: 1.0
ROUGE-1: 1.0, ROUGE-2: 1.0, ROUGE-L: 1.0
ROUGE-1: 0.9781021897810218, ROUGE-2: 0.962962962962963, ROUGE-L: 0.9781021897810218
ROUGE-1: 1.0, ROUGE-2: 1.0, ROUGE-L: 1.0
ROUGE-1: 1.0, ROUGE-2: 1.0, ROUGE-L: 1.0
ROUGE-1: 1.0, ROUGE-2: 1.0, ROUGE-L: 1.0


In [ ]:
pip install nltk


In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Example Usage:
for pred, true in zip(generated_answers, reference_answers):
    pred_tokens = pred.lower().split()
    true_tokens = [true.lower().split()]
    bleu_score = sentence_bleu(true_tokens, pred_tokens)
    print(f"BLEU Score: {bleu_score}")


BLEU Score: 0.1699212719881832
BLEU Score: 0.9691937043892331
BLEU Score: 1.0
BLEU Score: 0.9338603241255118
BLEU Score: 1.0
BLEU Score: 1.0
BLEU Score: 1.0


In [ ]:
# Combine all metrics in a single workflow
for pred, true in zip(generated_answers, reference_answers):
    # Exact Match & F1 Score
    em = exact_match(pred, true)
    _, _, f1 = precision_recall_f1(pred, true)

    # ROUGE Scores
    rouge = rouge_scores(pred, true)

    # BLEU Score
    pred_tokens = pred.lower().split()
    true_tokens = [true.lower().split()]
    bleu = sentence_bleu(true_tokens, pred_tokens)

    # Display Results
    print(f"Exact Match: {em}, F1: {f1}")
    print(f"ROUGE-1: {rouge['rouge1'].fmeasure}, ROUGE-2: {rouge['rouge2'].fmeasure}, ROUGE-L: {rouge['rougeL'].fmeasure}")
    print(f"BLEU Score: {bleu}")
    print("-" * 30)


Exact Match: 0, F1: 0.29629629629629634
ROUGE-1: 0.3157894736842105, ROUGE-2: 0.21818181818181817, ROUGE-L: 0.3157894736842105
BLEU Score: 0.1699212719881832
------------------------------
Exact Match: 0, F1: 0.8235294117647058
ROUGE-1: 1.0, ROUGE-2: 1.0, ROUGE-L: 1.0
BLEU Score: 0.9691937043892331
------------------------------
Exact Match: 1, F1: 0.84375
ROUGE-1: 1.0, ROUGE-2: 1.0, ROUGE-L: 1.0
BLEU Score: 1.0
------------------------------
Exact Match: 0, F1: 0.6956521739130436
ROUGE-1: 0.9781021897810218, ROUGE-2: 0.962962962962963, ROUGE-L: 0.9781021897810218
BLEU Score: 0.9338603241255118
------------------------------
Exact Match: 1, F1: 0.9090909090909091
ROUGE-1: 1.0, ROUGE-2: 1.0, ROUGE-L: 1.0
BLEU Score: 1.0
------------------------------
Exact Match: 1, F1: 0.9565217391304348
ROUGE-1: 1.0, ROUGE-2: 1.0, ROUGE-L: 1.0
BLEU Score: 1.0
------------------------------
Exact Match: 1, F1: 0.7051282051282052
ROUGE-1: 1.0, ROUGE-2: 1.0, ROUGE-L: 1.0
BLEU Score: 1.0
-----------------